<a href="https://colab.research.google.com/github/theroyalraj/Deep-Learning/blob/master/Traffic%20Volume%20Prediction/Traffic_Voulme_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras import backend as K
from keras.layers import Dense, Activation, Flatten,LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
from tensorflow import keras
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
from sklearn import preprocessing
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
train=pd.read_csv('/gdrive/My Drive/data/for_training.csv')

In [0]:
test=pd.read_csv('/gdrive/My Drive/data/for_testing.csv')

In [5]:
train.head()

,Unnamed: 0,time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
0,0,900,7,121,89,2,329,1,1,288.28,0.0,0.0,40,1,24
1,1,1000,7,178,67,3,330,1,1,289.36,0.0,0.0,75,1,2
2,2,1100,7,113,66,3,329,2,2,289.58,0.0,0.0,90,1,19
3,3,1200,7,20,66,3,329,5,5,290.13,0.0,0.0,90,1,19
4,4,1300,7,281,65,3,329,7,7,291.14,0.0,0.0,75,1,2


In [0]:
train.drop("Unnamed: 0",axis=1,inplace=True)

In [7]:
train.shape

(33750, 14)

In [8]:
test.head()

,time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
0,0,7,73,63,1,27,4,4,285.15,0.0,0,90,6,15
1,0,7,251,63,1,27,4,4,285.15,0.0,0,90,5,14
2,0,7,75,56,1,0,1,1,285.15,0.0,0,90,2,9
3,100,7,98,56,1,351,2,2,284.79,0.0,0,90,6,7
4,100,7,283,56,1,351,1,1,284.79,0.0,0,90,5,14


In [0]:
targets=pd.read_csv("/gdrive/My Drive/data/targets.csv",names=['traffic_volume'])

In [0]:
targets=targets.iloc[:,0]

In [0]:
std_scale = preprocessing.StandardScaler().fit(train)
x_train_norm = std_scale.transform(train)

In [11]:
targets.head()

0    5545
1    4516
2    4767
3    5026
4    4918
Name: traffic_volume, dtype: int64

In [0]:
std_scale = preprocessing.StandardScaler().fit(train)
x_train_norm = std_scale.transform(train)

In [0]:
trainX = np.reshape(x_train_norm, (x_train_norm.shape[0], 1,x_train_norm.shape[1]))

In [75]:
model = Sequential()

# The Input Layer :
model.add(LSTM(14, input_shape = (1,14),activation='relu'))

# The Hidden Layers :
model.add(Dense(256,kernel_initializer='glorot_normal',activation='relu'))
model.add(Dense(256, kernel_initializer='glorot_normal',activation='relu'))
model.add(Dense(256, kernel_initializer='glorot_normal',activation='relu'))
model.add(Dense(256, kernel_initializer='glorot_normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='glorot_normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 14)                1624      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               3840      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 203,097
Trainable params: 203,097
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 2, save_best_only = True, mode ='auto')
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
callbacks_list = [checkpoint,early_stop]

In [0]:
model.fit(trainX, targets, epochs=100, batch_size=30, validation_split = 0.1, callbacks=callbacks_list)

In [0]:
std_scale_test = preprocessing.StandardScaler().fit(test)
x_test_norm = std_scale.transform(test)

In [0]:
testX = np.reshape(x_test_norm, (x_test_norm.shape[0], 1,x_test_norm.shape[1]))

In [84]:
model.predict(testX)

array([[ 713.65674],
       [ 697.54034],
       [ 687.5874 ],
       ...,
       [2500.872  ],
       [1918.9572 ],
       [1247.2667 ]], dtype=float32)

In [0]:
wights_file = 'Weights-029--518.20289.hdf5' # choose the best checkpoint 
model.load_weights(wights_file) # load it
model.compile(loss='mean_absolute_error', optimizer='RMSprop', metrics=['mean_absolute_error'])

In [86]:
raw_test=pd.read_csv("/gdrive/My Drive/Colab Notebooks/Test.csv")
def make_submission(values, sub_name):
  my_submission = pd.DataFrame({'date_time':raw_test.date_time,'traffic_volume':values})
  my_submission.to_csv('/gdrive/My Drive/data/{}.csv'.format(sub_name),index=False)
  print('A submission file has been made')

values = model.predict(testX)
make_submission(values[:,0],'submit')

A submission file has been made


In [0]:
K.clear_session()

0.7319965167175